The main idea here is that I want to guess what are the most important keys ahead of time ()

In [1]:
# For boba

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

import torch
print(torch.cuda.device_count())

2


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16, attn_implementation='sdpa', device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

/home/vijay/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/vijay/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from typing import Optional, Tuple
from transformers.cache_utils import Cache
from transformers.models.llama.modeling_llama import apply_rotary_pos_emb, repeat_kv
import math
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from time import perf_counter

In [ ]:
import triton
import triton.language as tl
from triton_argsort import argsort

@triton.jit
def _triton_block_sparse_attn_scores_fwd_kernel(
    Q, K, seqlens,
    block_index,
    Out,
    stride_qz, stride_qh, stride_qm, stride_qk,
    stride_kz, stride_kh, stride_kn, stride_kk,
    stride_oz, stride_oh, stride_om, stride_on,
    Z, H, N_CTX,
    NUM_ROWS, MAX_BLOCKS_PRE_ROW,
    BLOCK_M: tl.constexpr,
    BLOCK_N: tl.constexpr,
    BLOCK_DMODEL: tl.constexpr,
    top_k: tl.constexpr,
    dtype: tl.constexpr,
):
    start_m = tl.program_id(0)
    off_hz = tl.program_id(1)

    seqlen = tl.load(seqlens + off_hz // H)
    if start_m * BLOCK_M >= seqlen:
        return

    # initialize offsets
    offs_m = start_m * BLOCK_M + tl.arange(0, BLOCK_M)
    offs_n = tl.arange(0, BLOCK_N)
    offs_d = tl.arange(0, BLOCK_DMODEL)

    qo_offset = (off_hz // H) * stride_qz + (off_hz % H) * stride_qh
    kv_offset = (off_hz // H) * stride_kz + (off_hz % H) * stride_kh

    q_ptrs = Q + qo_offset + offs_m[:, None] * stride_qm + offs_d[None, :] * stride_qk
    k_ptrs = K + kv_offset + offs_d[:, None] * stride_kk
    o_ptrs = Out + qo_offset + offs_m[:, None] * stride_om + offs_d[None, :] * stride_on

    blocks_ptr = block_index + (off_hz * NUM_ROWS + start_m) * MAX_BLOCKS_PRE_ROW

    # loop over k, v and update accumulator
    m_mask = offs_m[:, None] < seqlen
    block_count = tl.minimum((start_m + 1) * BLOCK_M // BLOCK_N, MAX_BLOCKS_PRE_ROW)

    # initialize the accumulator
    acc = tl.zeros([BLOCK_M, BLOCK_N * block_count], dtype=tl.float32) # block_m will be something small like 8, 
    indices = tl.zeros([BLOCK_M, BLOCK_N * block_count], dtype=tl.float32)
    # save the topk accumulator
    acc_topk = tl.zeros([BLOCK_M, top_k], dtype=tl.float32)

    # load q: it will stay in SRAM throughout
    q = tl.load(q_ptrs)
    q = q.to(dtype)

    for sparse_block_idx in range(block_count):
        real_block_idx = tl.load(blocks_ptr + sparse_block_idx)
        start_n = real_block_idx * BLOCK_N
        cols = start_n + offs_n
        # -- load k, v --
        k = tl.load(k_ptrs + cols[None, :] * stride_kn)
        # -- compute qk --
        qk = tl.zeros([BLOCK_M, BLOCK_N], dtype=tl.float32)
        # if start_n + BLOCK_N < seqlen:
        #     qk = tl.where(m_mask, qk, float("-inf"))
        # else:
        causal_mask = cols[None, :] <= offs_m[:, None]
        qk = tl.where(m_mask & causal_mask, qk, float("-inf"))
        qk += tl.dot(q, k)
        acc[:, sparse_block_idx * BLOCK_N : (sparse_block_idx + 1) * BLOCK_N] = qk
        indices[:, sparse_block_idx * BLOCK_N : (sparse_block_idx + 1) * BLOCK_N] = cols

    for idx in range(BLOCK_M):
        _, ids = argsort(acc[idx])
        tl.sort(ids)
        acc_topk[idx] = ids
    tl.store(o_ptrs, acc.to(dtype), mask=m_mask)

def _triton_block_sparse_attention_scores(
    q,                 # [BATCH, N_HEADS, N_CTX, D_HEAD]
    k,                 # [BATCH, N_HEADS, N_CTX, D_HEAD]
    seqlens,           # [BATCH, ]
    block_index,       # [BATCH, N_HEADS, cdiv(N_CTX, BLOCK_SIZE_M), MAX_BLOCKS_PRE_ROW]
    block_size_M=64,
    block_size_N=64,
) -> torch.Tensor:
    # shape constraints
    Lq, Lk = q.shape[-1], k.shape[-1]
    assert Lq == Lk
    assert Lk in {16, 32, 64, 128}

    B, H, Cq, D = q.shape
    top_count = block_index.shape[-1]

    o = torch.zeros((B, H, Cq, top_count * block_size_N), device=q.device)
    grid = (triton.cdiv(q.shape[2], block_size_M), q.shape[0] * q.shape[1], 1)
    dtype = tl.bfloat16 if q.dtype == torch.bfloat16 else tl.float16
    _triton_block_sparse_attn_scores_fwd_kernel[grid](
        q, k, seqlens, 
        block_index,
        o,
        q.stride(0), q.stride(1), q.stride(2), q.stride(3),
        k.stride(0), k.stride(1), k.stride(2), k.stride(3),
        o.stride(0), o.stride(1), o.stride(2), o.stride(3),
        q.shape[0], q.shape[1], q.shape[2],
        block_index.shape[-2], block_index.shape[-1],
        BLOCK_M=block_size_M, BLOCK_N=block_size_N,
        BLOCK_DMODEL=Lk,
        dtype=dtype,
        num_warps=4, num_stages=2,
    )

    return o

In [7]:
from typing import Optional
import einops

def _build_block_index(
    query: torch.Tensor,     # [BATCH, N_HEADS, N_CTX, D_HEAD]
    key: torch.Tensor,       # [BATCH, N_HEADS, N_CTX, D_HEAD]
    top_k: int,
    block_size_M: int = 64,
    block_size_N: int = 64,
):
    batch_size, num_heads, context_size, head_dim = query.shape
    query_pool = query.reshape((batch_size, num_heads, -1, block_size_M, head_dim)).mean(dim=-2)
    key_pool = key.reshape((batch_size, num_heads, -1, block_size_N, head_dim)).mean(dim=-2)
    arange_M = torch.arange(query_pool.shape[-2], dtype=torch.int32, device=query.device) * block_size_M
    arange_N = torch.arange(key_pool.shape[-2], dtype=torch.int32, device=key.device) * block_size_N
    p_pool = torch.einsum(f'bhmk, bhnk -> bhmn', query_pool, key_pool)
    p_pool = p_pool.where(arange_M[None, None, :, None] >= arange_N[None, None, None, :], -torch.inf)
    top_k = min(top_k, context_size // block_size_N)
    return torch.topk(p_pool, top_k, dim=-1).indices.to(torch.int32).sort(dim=-1).values

def _build_block_index_subselect(
    query: torch.Tensor,     # [BATCH, N_HEADS, N_CTX, D_HEAD]
    key: torch.Tensor,       # [BATCH, N_HEADS, N_CTX, D_HEAD]
    top_k: int,
    block_size_M: int = 32,
    block_size_N: int = 32,
    subselect_scale: int = 8
):
    batch_size, num_heads, context_size, head_dim = query.shape
    query_pool = query.reshape((batch_size, num_heads, -1, block_size_M, head_dim)).mean(dim=-2)
    key_pool = key.reshape((batch_size, num_heads, -1, block_size_N, head_dim)).mean(dim=-2)

    query_pool_subselect = query.reshape((batch_size, num_heads, -1, subselect_scale, head_dim)).mean(dim=-2)
    key_pool_subselect = key.reshape((batch_size, num_heads, -1, subselect_scale, head_dim)).mean(dim=-2)
    subselect_block_index = _build_block_index(query_pool, key_pool, top_k * 2, subselect_scale, subselect_scale) # [BATCH, N_HEADS, N_CTX // BLOCK_SIZE_M, TOP_K]
    seqlens = torch.tensor([context_size], dtype=torch.int32, device=query.device)
    attention_scores = _triton_block_sparse_attention_scores(
        query_pool, key_pool, seqlens, subselect_block_index, subselect_scale, subselect_scale 
    ) # each index here defines a different affinity with a block of subselect_scale

    # Don't need to add masking here because it was done while collecting attention scores
    p_pool = torch.topk(attention_scores, top_k, dim=-1).indices.to(torch.int32).sort(dim=-1).values
    
    # repeat and turn it into a series of blocks that then need to be gathered
    # let's just say that the scores have been identified

    top_k = min(top_k, context_size // block_size_N)
    return torch.topk(p_pool, top_k, dim=-1).indices.to(torch.int32).sort(dim=-1).values


def block_sparse_attention(
    query: torch.Tensor,  # [BATCH, N_HEADS, N_CTX, D_HEAD]
    key: torch.Tensor,    # [BATCH, N_HEADS, N_CTX, D_HEAD]
    value: torch.Tensor,  # [BATCH, N_HEADS, N_CTX, D_HEAD]
    top_k: int,
    block_size_M: int = 64,
    block_size_N: int = 64,
):
    batch_size, num_heads, context_size, head_dim = query.shape
    pad = block_size_M - (query.shape[2] & (block_size_M - 1))
    query = torch.nn.functional.pad(query, [0, 0, 0, pad, 0, 0, 0, 0])
    key = torch.nn.functional.pad(key, [0, 0, 0, pad, 0, 0, 0, 0])
    value = torch.nn.functional.pad(value, [0, 0, 0, pad, 0, 0, 0, 0])
    seqlens = torch.tensor([context_size], dtype=torch.int32, device=query.device)
    sm_scale = head_dim ** -0.5
    block_index = _build_block_index(query, key, top_k, block_size_N, block_size_N)
    out = _triton_block_sparse_attention(query, key, value, seqlens, block_index, sm_scale, block_size_M, block_size_N)
    return out[..., :context_size, :]

In [8]:
def sdpa_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_value: Optional[Cache] = None,
    output_attentions: bool = False,
    use_cache: bool = False,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    # print("Running custom forward function")
    bsz, q_len, _ = hidden_states.size()

    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None:
        kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
    cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)

    query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

    if past_key_value is not None:
        cache_kwargs = {"sin": sin, "cos": cos}  # Specific to RoPE models
        key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)

    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    if attention_mask is not None:
        if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
            raise ValueError(
                f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
            )

    # SDPA with memory-efficient backend is currently (torch==2.1.2) bugged with non-contiguous inputs with custom attn_mask,
    # Reference: https://github.com/pytorch/pytorch/issues/112577.
    if query_states.device.type == "cuda" and attention_mask is not None:
        query_states = query_states.contiguous()
        key_states = key_states.contiguous()
        value_states = value_states.contiguous()

    # generate weights for which tokens are important or not

    # Apply padding to q and k
    max_block_size = 32
    k_len = key_states.shape[-2]
    pad_len_q = 0 if (q_len % max_block_size) == 0 else (max_block_size - (q_len % max_block_size))
    pad_len_k = 0 if (k_len % max_block_size) == 0 else (max_block_size - (k_len % max_block_size))
    if pad_len_q > 0:
        bq, hq, tq, dq = query_states.shape
        pad_matrix = torch.zeros((bq, hq, pad_len_q, dq), device=query_states.device)
        query_states = torch.cat((query_states, pad_matrix), dim=-2)  # Padding on the last dimension
    if pad_len_k > 0:
        bk, hk, tk, dk = key_states.shape
        pad_matrix = torch.zeros((bk, hk, pad_len_k, dk), device=query_states.device)
        key_states = torch.cat((key_states, pad_matrix), dim=-2)  # Padding on the last dimension
        value_states = torch.cat((value_states, pad_matrix), dim=-2)
    print("Padding length: ", pad_len_q, pad_len_k)
    q_blocks, k_blocks, v_blocks = low_dim_get_attention_scores(query_states, key_states, value_states)

    attn_output = torch.nn.functional.scaled_dot_product_attention(
        q_blocks,
        k_blocks,
        v_blocks,
        attn_mask=attention_mask,
        dropout_p=self.attention_dropout if self.training else 0.0,
        # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
        is_causal=self.is_causal and attention_mask is None and q_len > 1,
    )

    # TODO: rejoin the output to fit the desired size.

    attn_output = attn_output.transpose(1, 2).contiguous()
    attn_output = attn_output.reshape(bsz, q_len, self.hidden_size) # q_len is padded to be of the same length for everything

    attn_output = self.o_proj(attn_output)

    return attn_output, None, past_key_value

for i in range(len(model.model.layers)):
    model.model.layers[i].self_attn.forward = sdpa_forward.__get__(model.model.layers[i].self_attn, type(model.model.layers[i].self_attn))

In [9]:
sample_text = open("snapkv_full.txt", "r", encoding="utf-8").read()
encoded_tokens = tokenizer(sample_text, return_tensors="pt")
for key in encoded_tokens:
    encoded_tokens[key] = encoded_tokens[key].cuda()

# find a way to start by chunking this portion

# there shouuld be a mix between the global attention and the attention that only the last few tokens (query) pay

past_key_values = None
outputs = model.generate(**encoded_tokens, output_attentions=True, return_dict_in_generate=True, max_new_tokens=1)

Padding length:  15 15
Queries shape:  torch.Size([1, 32, 17600, 128])  Keys shape:  torch.Size([1, 32, 17600, 128])  Values shape:  torch.Size([1, 32, 17600, 128])
Q chunks shape:  torch.Size([32, 550, 128])  K chunks shape:  torch.Size([32, 550, 128])
Lower dim matrix shapes:  torch.Size([32, 550, 32]) torch.Size([32, 550, 32])
Attention map:  torch.Size([32, 550, 550])
Top block count:  137
Top blocks shape:  torch.Size([32, 550, 137])
Top blocks:  tensor([[[  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         ...,
         [  0,   1,   2,  ..., 134, 135, 547],
         [  0,   1,   2,  ..., 134, 135, 548],
         [  0,   1,   2,  ..., 134, 547, 548]],

        [[  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         ...,
         [  0,   1,   2,  ..., 324, 325, 547],
         [  0,   1,   2,  ..., 325, 547, 548

OutOfMemoryError: CUDA out of memory. Tried to allocate 36.79 GiB. GPU 0 has a total capacity of 23.69 GiB of which 12.03 GiB is free. Including non-PyTorch memory, this process has 11.65 GiB memory in use. Of the allocated memory 10.31 GiB is allocated by PyTorch, and 1.03 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)